In [20]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

# CONSTS

In [21]:
CHROME_DRIVER_PATH = "./chromedriver-mac/chromedriver"
BRAVE_PATH = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
DOWNLOAD_DIRECTORY = "/Users/mamoudou/Developer/Python/ProjetTrutoré/"
ARCHIVE_DIRECTORY = "/Users/mamoudou/Developer/Python/ProjetTrutoré/archive"
URL = "https://donneespubliques.meteofrance.fr/?fond=produit&id_produit=90&id_rubrique=32"

## Browser Configs

In [22]:
options = Options()
options.binary_location = BRAVE_PATH  

prefs = {
    "download.default_directory": DOWNLOAD_DIRECTORY,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True 
}
options.add_experimental_option("prefs", prefs)

In [23]:
service = Service(executable_path=CHROME_DRIVER_PATH)

driver = webdriver.Chrome(service=service, options=options)



driver.get(URL)
print("Page Title:", driver.title)
telecharger_button = driver.find_element(By.XPATH, "//input[@type='submit' and @value='Télécharger']")
telecharger_button.click()

time.sleep(5)
driver.quit()

Page Title: Données Publiques de Météo-France - Données SYNOP essentielles OMM


## Fixing the file extension problem

In [24]:
from file_management import change_file_extension

cvs_file = change_file_extension(DOWNLOAD_DIRECTORY)

Le fichier a été renommé : /Users/mamoudou/Developer/Python/ProjetTrutoré/synop.2025010518.txt -> /Users/mamoudou/Developer/Python/ProjetTrutoré/synop.2025010518.csv


# Starting the ELT

In [25]:
import pandas as pd
import numpy as np
df = pd.read_csv(cvs_file, sep=";", dtype={"date": str})

print(df.to_string())

    numer_sta            date    pmer  tend cod_tend   dd    ff           t          td    u     vv  ww  w1  w2    n nbas  hbas  cl  cm  ch    pres niv_bar geop tend24        tn12        tn24        tx12        tx24     tminsol  sw  tw      raf10     rafper  per etat_sol  ht_neige    ssfrai perssfrai        rr1        rr3        rr6       rr12       rr24 phenspe1 phenspe2 phenspe3 phenspe4 nnuage1 ctype1 hnuage1 nnuage2 ctype2 hnuage2 nnuage3 ctype3 hnuage3 nnuage4 ctype4 hnuage4  Unnamed: 59
0        7005  20250105180000   99110  -100        8  190   2.5  284.650000  283.250000   91  20000   0  mq  mq   mq   mq    mq  mq  mq  mq   98260      mq   mq  -1980  283.050000          mq  285.350000          mq  278.650000  mq  mq   8.700000  10.100000  -10        1  0.000000        mq        mq   0.200000   0.800000   0.800000   4.200000  14.100000       mq       mq       mq       mq      mq     mq      mq      mq     mq      mq      mq     mq      mq      mq     mq      mq          NaN
1   

# Récupération des informations importantes à savoir :
1. Indicatif OMM station : numéro de station `numer_sta` -> `int`
2. Date (UTC) : `date` -> `datetime`
3. Pression au niveau mer : `pmer` -> `int`
4. Variation de pression en 3 heures : `tend` -> `int`
5. Type de tendance barométrique : `cod_tend` -> `int`
6. Direction du vent moyen 10 mn : `dd` -> `int`
7. Vitesse du vent moyen 10 mn : `ff` -> `float`
8. Température : `t` -> `float`
9. Point de rosée : `td` -> `float`
10. Humidité : `u` -> `int`
11. Visibilité horizontale : `vv` -> `float`
12. Temps présent : `ww` -> `int`
13. Nébulosité totale : `n` -> `float`
14. Nébulosité des nuages de l'étage inférieur : `nbas` -> `int`
15. Hauteur de la base des nuages de l'étage inférieur : `hbas` -> `int`
16. Pression station : `pres` -> `int`
17. Variation de pression en 24 heures : `tend24` -> `int`
18. Température minimale sur N heures : `tn12` -> `float`
19. Température maximale sur N heures : `tx12` -> `float`
20. Température minimale du sol sur 12 heures : `tminsol` -> `float`
21. Rafales sur les 10 dernières minutes : `raf10` -> `float`
22. Rafales sur une période : `rafper` -> `float`
23. Période de mesure des rafales : `per` -> `float`
24. Précipitations dans les N dernières heures : `rr12` -> `float`

In [26]:
df = df[['numer_sta', 'date', "pmer", "tend", "cod_tend", "dd", "ff", "t", "td", "u", "vv", "ww", "n", "nbas", "hbas","pres", "tend24", "tn12", "tx12", "tminsol", "raf10", "rafper", "per", "rr12" ]]


print(df.to_string())


    numer_sta            date    pmer  tend cod_tend   dd    ff           t          td    u     vv  ww    n nbas  hbas    pres tend24        tn12        tx12     tminsol      raf10     rafper  per       rr12
0        7005  20250105180000   99110  -100        8  190   2.5  284.650000  283.250000   91  20000   0   mq   mq    mq   98260  -1980  283.050000  285.350000  278.650000   8.700000  10.100000  -10   4.200000
1        7015  20250105180000   99150   -60        7  190   8.9  284.050000  282.950000   93  16250  61  100    8   450   98590  -2150  275.250000  285.550000  274.650000  14.200000  14.200000  -10   3.800000
2        7020  20250105180000   98640  -190        8  180  11.9  285.850000  285.050000   95     mq  mq   mq   mq    mq   98530  -1780  285.050000  285.950000          mq  17.700000  19.000000  -10   4.800000
3        7027  20250105180000   99050  -150        8  180   7.8  285.550000  284.150000   91  12630   0  100    6   450   98260  -1630  285.350000  286.350000  280.

## Remplacer les valeurs manquantes `mq` par `NaN`

In [27]:
df.replace("mq", np.nan, inplace=True)
print(df.to_string())

    numer_sta            date    pmer  tend cod_tend   dd    ff           t          td    u     vv   ww    n nbas  hbas    pres tend24        tn12        tx12     tminsol      raf10     rafper  per       rr12
0        7005  20250105180000   99110  -100        8  190   2.5  284.650000  283.250000   91  20000    0  NaN  NaN   NaN   98260  -1980  283.050000  285.350000  278.650000   8.700000  10.100000  -10   4.200000
1        7015  20250105180000   99150   -60        7  190   8.9  284.050000  282.950000   93  16250   61  100    8   450   98590  -2150  275.250000  285.550000  274.650000  14.200000  14.200000  -10   3.800000
2        7020  20250105180000   98640  -190        8  180  11.9  285.850000  285.050000   95    NaN  NaN  NaN  NaN   NaN   98530  -1780  285.050000  285.950000         NaN  17.700000  19.000000  -10   4.800000
3        7027  20250105180000   99050  -150        8  180   7.8  285.550000  284.150000   91  12630    0  100    6   450   98260  -1630  285.350000  286.350000 

## Nettoyages de colonnes non renseignées 

1. Informations générales
	-	Numéro de station : `numer_sta` -> int
	-	Date (UTC) : `date` -> datetime
2. Pression
	-	Pression au niveau mer : `pmer` -> int
	-	Variation de pression en 3 heures : `tend` -> int
	-	Type de tendance barométrique : `cod_tend` -> int
	-	Pression station : `pres` -> int
	-	Variation de pression en 24 heures : `tend24` -> int
3. Vent
	-	Direction du vent moyen 10 mn : `dd` -> int
	-	Vitesse du vent moyen 10 mn : `ff` -> float
	-	Rafales sur les 10 dernières minutes : `raf10` -> float
	-	Rafales sur une période : `rafper` -> float
	-	Période de mesure des rafales : `per` -> float
4. Température
	-	Température : `t` -> float
	-	Point de rosée : `td` -> float
	-	Température minimale sur N heures : `tn12` -> float
	-	Température maximale sur N heures : `tx12` -> float
	-	Température minimale du sol sur 12 heures : `tminsol` -> float
5. Humidité
	-	Humidité : `u` -> int
6. Visibilité
	-	Visibilité horizontale : `vv` -> float
7. Nébulosité et nuages
	-	Nébulosité totale : `n` -> float
	-	Nébulosité des nuages de l’étage inférieur : `nbas` -> int
	-	Hauteur de la base des nuages de l’étage inférieur : `hbas` -> int
8. Précipitations
	-	Précipitations dans les N dernières heures : `rr12` -> float
9. Temps présent
	-	Temps présent : `ww` -> int

In [28]:
df[['dd', 'ff', 'raf10', 'rafper', 'rr12', "per"]] = df[['dd', 'ff', 'raf10', 'rafper', 'rr12', "per"]].fillna(0)

moyenne = ["pmer", "tend", "cod_tend", "pres", "tend24", "t", "td", "tn12", "tx12", "tminsol", "u", "vv", "n", "nbas", "hbas", "ww"]
df[moyenne] = df[moyenne].apply(pd.to_numeric, errors='coerce')

for col in moyenne:
    df[col] = df[col].fillna(df[col].mean())

print(df.to_string())

    numer_sta            date           pmer       tend  cod_tend   dd    ff           t          td          u            vv         ww           n      nbas    hbas           pres       tend24        tn12        tx12     tminsol      raf10     rafper  per       rr12
0        7005  20250105180000   99110.000000 -100.00000  8.000000  190   2.5  284.650000  283.250000   91.00000  20000.000000   0.000000   94.517241  5.894737  2250.0   98260.000000 -1980.000000  283.050000  285.350000  278.650000   8.700000  10.100000  -10   4.200000
1        7015  20250105180000   99150.000000  -60.00000  7.000000  190   8.9  284.050000  282.950000   93.00000  16250.000000  61.000000  100.000000  8.000000   450.0   98590.000000 -2150.000000  275.250000  285.550000  274.650000  14.200000  14.200000  -10   3.800000
2        7020  20250105180000   98640.000000 -190.00000  8.000000  180  11.9  285.850000  285.050000   95.00000  25125.909091   8.488372   94.517241  5.894737  2250.0   98530.000000 -1780.00000

## Typages des colonnes

In [29]:
df["date"] = pd.to_datetime(df["date"], format='%Y%m%d%H%M%S')
df['date'] = df['date'].dt.strftime("%Y-%m-%d %H:%M:%S")

entier = ["numer_sta", "pmer", "tend", "cod_tend", "dd", "u", "ww", "nbas", "hbas", "pres", "tend24"]
for i in entier:
    print(f"{i} -> {df[i].dtype}")
    df[i] = df[i].astype(int)
    print(f"{i} -> {df[i].dtype}")
    print("")

double = ['dd', 'ff', 'raf10', 'rafper', 'rr12', "per", "t", "td", "tn12", "tx12", "tminsol", "n", "vv"]
for i in double:
    print(f"{i} -> {df[i].dtype}")
    df[i] = df[i].astype(float)
    print(f"{i} -> {df[i].dtype}")
    print("")

numer_sta -> int64
numer_sta -> int64

pmer -> float64
pmer -> int64

tend -> float64
tend -> int64

cod_tend -> float64
cod_tend -> int64

dd -> int64
dd -> int64

u -> float64
u -> int64

ww -> float64
ww -> int64

nbas -> float64
nbas -> int64

hbas -> float64
hbas -> int64

pres -> float64
pres -> int64

tend24 -> float64
tend24 -> int64

dd -> int64
dd -> float64

ff -> float64
ff -> float64

raf10 -> object
raf10 -> float64

rafper -> object
rafper -> float64

rr12 -> object
rr12 -> float64

per -> object
per -> float64

t -> float64
t -> float64

td -> float64
td -> float64

tn12 -> float64
tn12 -> float64

tx12 -> float64
tx12 -> float64

tminsol -> float64
tminsol -> float64

n -> float64
n -> float64

vv -> float64
vv -> float64



In [30]:
print(df.to_string())

    numer_sta                 date    pmer  tend  cod_tend     dd    ff           t          td    u            vv  ww           n  nbas  hbas    pres  tend24        tn12        tx12     tminsol  raf10  rafper   per  rr12
0        7005  2025-01-05 18:00:00   99110  -100         8  190.0   2.5  284.650000  283.250000   91  20000.000000   0   94.517241     5  2250   98260   -1980  283.050000  285.350000  278.650000    8.7    10.1 -10.0   4.2
1        7015  2025-01-05 18:00:00   99150   -60         7  190.0   8.9  284.050000  282.950000   93  16250.000000  61  100.000000     8   450   98590   -2150  275.250000  285.550000  274.650000   14.2    14.2 -10.0   3.8
2        7020  2025-01-05 18:00:00   98640  -190         8  180.0  11.9  285.850000  285.050000   95  25125.909091   8   94.517241     5  2250   98530   -1780  285.050000  285.950000  280.361111   17.7    19.0 -10.0   4.8
3        7027  2025-01-05 18:00:00   99050  -150         8  180.0   7.8  285.550000  284.150000   91  12630.0000

## Conversion des températures de Kelvin en Celcuis

In [31]:
celcuis = ["t", "td", "tn12", "tx12", "tminsol"]
for i in celcuis:
    df[i] = df[i] - 273.15

print(df.to_string())

    numer_sta                 date    pmer  tend  cod_tend     dd    ff          t         td    u            vv  ww           n  nbas  hbas    pres  tend24       tn12       tx12    tminsol  raf10  rafper   per  rr12
0        7005  2025-01-05 18:00:00   99110  -100         8  190.0   2.5  11.500000  10.100000   91  20000.000000   0   94.517241     5  2250   98260   -1980   9.900000  12.200000   5.500000    8.7    10.1 -10.0   4.2
1        7015  2025-01-05 18:00:00   99150   -60         7  190.0   8.9  10.900000   9.800000   93  16250.000000  61  100.000000     8   450   98590   -2150   2.100000  12.400000   1.500000   14.2    14.2 -10.0   3.8
2        7020  2025-01-05 18:00:00   98640  -190         8  180.0  11.9  12.700000  11.900000   95  25125.909091   8   94.517241     5  2250   98530   -1780  11.900000  12.800000   7.211111   17.7    19.0 -10.0   4.8
3        7027  2025-01-05 18:00:00   99050  -150         8  180.0   7.8  12.400000  11.000000   91  12630.000000   0  100.000000    

## Arrondir tous les decimaux à 2chiffres apes la virgules

In [32]:
df[double] = df[double].round(2)
print(df.to_string())

    numer_sta                 date    pmer  tend  cod_tend     dd    ff      t     td    u        vv  ww       n  nbas  hbas    pres  tend24   tn12   tx12  tminsol  raf10  rafper   per  rr12
0        7005  2025-01-05 18:00:00   99110  -100         8  190.0   2.5  11.50  10.10   91  20000.00   0   94.52     5  2250   98260   -1980   9.90  12.20     5.50    8.7    10.1 -10.0   4.2
1        7015  2025-01-05 18:00:00   99150   -60         7  190.0   8.9  10.90   9.80   93  16250.00  61  100.00     8   450   98590   -2150   2.10  12.40     1.50   14.2    14.2 -10.0   3.8
2        7020  2025-01-05 18:00:00   98640  -190         8  180.0  11.9  12.70  11.90   95  25125.91   8   94.52     5  2250   98530   -1780  11.90  12.80     7.21   17.7    19.0 -10.0   4.8
3        7027  2025-01-05 18:00:00   99050  -150         8  180.0   7.8  12.40  11.00   91  12630.00   0  100.00     6   450   98260   -1630  12.20  13.20     7.70   12.4    13.2 -10.0   3.2
4        7037  2025-01-05 18:00:00   99290   

# Archive file

In [33]:
from file_management import move_file

move_file(cvs_file, ARCHIVE_DIRECTORY)

Le fichier a été déplacé de /Users/mamoudou/Developer/Python/ProjetTrutoré/synop.2025010518.csv à /Users/mamoudou/Developer/Python/ProjetTrutoré/archive/synop.2025010518.csv


## Converting DF to JSON and then to readable dict

In [34]:
from datetime import datetime

In [35]:
import pprint   
import json

json_data = df.to_json(orient='records')
dict_data = json.loads(json_data)

pprint.pprint(dict_data)

[{'cod_tend': 8,
  'date': '2025-01-05 18:00:00',
  'dd': 190.0,
  'ff': 2.5,
  'hbas': 2250,
  'n': 94.52,
  'nbas': 5,
  'numer_sta': 7005,
  'per': -10.0,
  'pmer': 99110,
  'pres': 98260,
  'raf10': 8.7,
  'rafper': 10.1,
  'rr12': 4.2,
  't': 11.5,
  'td': 10.1,
  'tend': -100,
  'tend24': -1980,
  'tminsol': 5.5,
  'tn12': 9.9,
  'tx12': 12.2,
  'u': 91,
  'vv': 20000.0,
  'ww': 0},
 {'cod_tend': 7,
  'date': '2025-01-05 18:00:00',
  'dd': 190.0,
  'ff': 8.9,
  'hbas': 450,
  'n': 100.0,
  'nbas': 8,
  'numer_sta': 7015,
  'per': -10.0,
  'pmer': 99150,
  'pres': 98590,
  'raf10': 14.2,
  'rafper': 14.2,
  'rr12': 3.8,
  't': 10.9,
  'td': 9.8,
  'tend': -60,
  'tend24': -2150,
  'tminsol': 1.5,
  'tn12': 2.1,
  'tx12': 12.4,
  'u': 93,
  'vv': 16250.0,
  'ww': 61},
 {'cod_tend': 8,
  'date': '2025-01-05 18:00:00',
  'dd': 180.0,
  'ff': 11.9,
  'hbas': 2250,
  'n': 94.52,
  'nbas': 5,
  'numer_sta': 7020,
  'per': -10.0,
  'pmer': 98640,
  'pres': 98530,
  'raf10': 17.7,
  'rafp

{"numer_sta": 7005, "date": "2025-01-05 18:00:00", "pmer": 99110, "tend": -100, "cod_tend": 8, "dd": 190.0, "ff": 2.5, "t": 11.5, "td": 10.1, "u": 91, "vv": 20000.0, "ww": 0, "n": 94.52, "nbas": 5, "hbas": 2250, "pres": 98260, "tend24": -1980, "tn12": 9.9, "tx12": 12.2, "tminsol": 5.5, "raf10": 8.7, "rafper": 10.1, "per": -10.0, "rr12": 4.2}


In [37]:
import requests
from api import send_data

In [41]:
first = dict_data[0]
js = json.dumps(first)

print(js)

url_api = "https://127.0.0.1:8000/product" 
donnees = {
    "name": "Essai",
    "description": "Superbe desc",
    "price": 122.2,
    'createdAt': '2024-12-22T16:44:44+00:00',
    'updatedAt': '2024-12-22T16:44:44+00:00'
}

send_data(url_api, donnees, requests)


{"numer_sta": 7005, "date": "2025-01-05 18:00:00", "pmer": 99110, "tend": -100, "cod_tend": 8, "dd": 190.0, "ff": 2.5, "t": 11.5, "td": 10.1, "u": 91, "vv": 20000.0, "ww": 0, "n": 94.52, "nbas": 5, "hbas": 2250, "pres": 98260, "tend24": -1980, "tn12": 9.9, "tx12": 12.2, "tminsol": 5.5, "raf10": 8.7, "rafper": 10.1, "per": -10.0, "rr12": 4.2}
Requête réussie!


[{'id': 16,
  'name': 'Essai',
  'description': 'Superbe desc',
  'price': 122.2,
  'createdAt': '2024-12-22T16:44:44+00:00',
  'updatedAt': '2024-12-22T16:44:44+00:00'}]

In [ ]:
URL_API = "https://127.0.0.1:8080/api/donnees_climatiques" 
for i in dict_data:
    data = json.dumps(i)
    send_data(URL_API, data, requests)